In [1]:
from pathlib import Path
from course_intro_ocr_t1.data import MidvPackage
from tqdm import tqdm
from matplotlib import pyplot as plt
import numpy as np


from shapely.geometry import Polygon

In [2]:
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn

from torchvision.models import resnet50
from torch.utils.data import Dataset, DataLoader

from torch.utils.tensorboard import SummaryWriter

import cv2

/home/tokyo-jazz/.local/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/tokyo-jazz/.local/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


Предполагаем, что датасет скачен в текущую директорию, датасет - https://github.com/fcakyon/midv500

In [3]:
DATASET_PATH = Path().absolute() / 'midv500_data' / 'midv500'
assert DATASET_PATH.exists(), DATASET_PATH.absolute()

In [4]:
# Собираем список пакетов (MidvPackage) 
data_packs = MidvPackage.read_midv500_dataset(DATASET_PATH)
len(data_packs), type(data_packs[0])

(50, course_intro_ocr_t1.data.MidvPackage)

Явно разделим на тест/трейн для использования Dataloader

In [5]:
train_indices = []
test_indices = []

for pack_idx in tqdm(range(len(data_packs))):
    for item_idx in range(len(data_packs[pack_idx])):
        if data_packs[pack_idx][item_idx].is_test_split():
            test_indices.append((pack_idx, item_idx))
        else:
            train_indices.append((pack_idx, item_idx))

100%|██████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 583.84it/s]


В финальной версии я добавил всю валидацию в трейн (конечно, до этого запуски проводились с валидацией)

In [6]:
# train, val = train_test_split(train_indices, test_size=0.1, random_state=42)

Преобразуем картинки к одинаковому размеру путем

1) паддинга с краев средними элементами до квадратного размера

2) resize к размеру `224x224`, как принято в resnet

3) нормализации `x -> (x - 127.5) / 255` в предположении что среднее картинок примерно `127.5` 

In [7]:
class ToSquare(nn.Module):
    def forward(self, img, x1, y1, x2, y2, x3, y3, x4, y4):
        h, w, _ = img.shape
        return (np.pad(
                img, 
                (
                    (max(0, (w - h) // 2), max(0, (w - h + 1) // 2)),
                    (max(0, (h - w) // 2), max(0, (h - w + 1) // 2)),
                    (0, 0)
                ),
                constant_values=127.5
            ), 
            x1 + max(0, (h - w) // 2), 
            y1 + max(0, (w - h) // 2), 
            x2 + max(0, (h - w) // 2), 
            y2 + max(0, (w - h) // 2), 
            x3 + max(0, (h - w) // 2), 
            y3 + max(0, (w - h) // 2), 
            x4 + max(0, (h - w) // 2), 
            y4 + max(0, (w - h) // 2)
        )

class ToSquareSize(nn.Module):
    def __init__(self, size):
        super().__init__()
        self.size = size
        
    def forward(self, img, x1, y1, x2, y2, x3, y3, x4, y4):
        img, x1, y1, x2, y2, x3, y3, x4, y4 = ToSquare()(img, x1, y1, x2, y2, x3, y3, x4, y4)
        cur_size = img.shape[0]
        img = cv2.resize(img, (self.size, self.size))
        x1 = (x1 * self.size / cur_size)
        y1 = (y1 * self.size / cur_size)
        x2 = (x2 * self.size / cur_size)
        y2 = (y2 * self.size / cur_size)
        x3 = (x3 * self.size / cur_size)
        y3 = (y3 * self.size / cur_size)
        x4 = (x4 * self.size / cur_size)
        y4 = (y4 * self.size / cur_size)
        return img, x1, y1, x2, y2, x3, y3, x4, y4

In [8]:
class MyDataset(Dataset):
    def __init__(self, ids):
        super().__init__()
        self.ids = ids
    
    def __len__(self):
        return len(self.ids)
    
    def __getitem__(self, idx):
        fi, si = self.ids[idx]
        di = data_packs[fi][si]
        im = np.array(di.image)
        im, x1, y1, x2, y2, x3, y3, x4, y4 = ToSquareSize(224)(
            im,
            *di.gt_data['quad'][0],
            *di.gt_data['quad'][1],
            *di.gt_data['quad'][2],
            *di.gt_data['quad'][3]
        )
        im = (im - 127.5) / 255
        return torch.tensor(im).permute(2, 0, 1), torch.tensor([x1, y1, x2, y2, x3, y3, x4, y4])

In [9]:
# train_dataset = MyDataset(train)
# val_dataset = MyDataset(val)

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, drop_last=True, num_workers=8)
# val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, drop_last=True, num_workers=8)

In [10]:
train_dataset = MyDataset(train_indices)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, drop_last=True, num_workers=8)

In [11]:
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.resnet = resnet50()
        self.resnet.fc = nn.Linear(2048, 8)
        
    def forward(self, x):
        return self.resnet(x)

In [12]:
model = MyModel()
assert torch.cuda.is_available()
device = torch.device('cuda')
model.to(device)

MyModel(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (

In [13]:
class MyLoss(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, inputs, targets):
        ious = []
        for idx in range(len(inputs)):
            pred_poly = Polygon((
                (inputs[idx][0], inputs[idx][1]),
                (inputs[idx][2], inputs[idx][3]),
                (inputs[idx][4], inputs[idx][5]),
                (inputs[idx][6], inputs[idx][7])
            ))
            target_poly = Polygon((
                (targets[idx][0], targets[idx][1]),
                (targets[idx][2], targets[idx][3]),
                (targets[idx][4], targets[idx][5]),
                (targets[idx][6], targets[idx][7])
            ))
            try:
                ious.append(pred_poly.intersection(target_poly).area / pred_poly.union(target_poly).area)
            except:
                ious.append(0)
        return torch.tensor(ious)

In [14]:
optim = torch.optim.AdamW(model.parameters(), lr=3e-4)
loss_fn = nn.MSELoss()

In [15]:
def run_validation(val_loader: DataLoader, model: nn.Module, n_steps=None):
    model.eval()
    n_good = 0
    n_all = 0
    wrapper = lambda x: x
    if n_steps is None:
        n_steps = len(val_loader)
        wrapper = tqdm
    
    
    ls = []
    
    with torch.no_grad():
        for batch, (X, y) in enumerate(wrapper(val_loader)):
            if batch == n_steps:
                break
            logits = model(X.to(torch.float32).to(device))
            ls.append(loss_fn(logits, y.to(device)).item())
            ious = MyLoss()(logits.to('cpu'), y.to('cpu')) 
            n_good += sum([1 if x > 0.95 else 0 for x in ious])
            n_all += len(ious)
    
    print("mse: ", np.mean(ls))
    return n_good / n_all


def train_epoch(train_loader: DataLoader, val_loader: DataLoader, model: nn.Module, optim, loss_fn):
    for batch, (X, y) in enumerate(tqdm(train_loader)):
        model.train()
        logits = model(X.to(torch.float32).to(device))
        loss = loss_fn(logits, y.to(torch.float32).to(device))
        
        if batch % 100 == 0:
            tb.add_scalar("losses/train_loss", loss, global_step=epoch*len(train_loader)+batch)
            ious = MyLoss()(logits, y) 
            tb.add_scalar("losses/train_acc", sum([1 if x > 0.95 else 0 for x in ious])/len(ious), global_step=epoch*len(train_loader)+batch)
            tb.add_scalar("losses/train_iou", torch.mean(ious), global_step=epoch*len(train_loader)+batch)
        
        optim.zero_grad()
        loss.backward()
        optim.step()

In [ ]:
tb = SummaryWriter()

for epoch in range(1000):
    print(f'Epoch {epoch}:')
    train_epoch(train_loader, None, model, optim, loss_fn)
    if epoch % 5 == 0:
        torch.save(model, f"resnet50-epoch{epoch}.ckpt")

Epoch 0:


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 335/335 [01:43<00:00,  3.23it/s]


Epoch 1:


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 335/335 [01:43<00:00,  3.25it/s]


Epoch 2:


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 335/335 [01:43<00:00,  3.23it/s]


Epoch 60:


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 335/335 [01:43<00:00,  3.23it/s]


Epoch 61:


 90%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏               | 303/335 [01:33<00:09,  3.26it/s]

p.s. вывод в ноутбук отваливается примерно через 1.5 часа с ошибкой, аналогочной https://github.com/tensorflow/tensorflow/issues/60309, но само обучение продолжается...

# Test

In [15]:
model = torch.load('checkpoints/resnet50-epoch350.ckpt')
model.to('cuda')

MyModel(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (

In [16]:
test_dataset = MyDataset(test_indices)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=8)

In [17]:
preds = []
model.eval()
with torch.no_grad():
    for X, _ in tqdm(test_loader):
        preds.extend(model(X.to(torch.float32).cuda()).detach().cpu().numpy())
preds = np.array(preds)

100%|█████████████████████████████████████████████████████████████████████████████| 266/266 [00:24<00:00, 10.67it/s]


Нужно откатить наше преобразование к квадрату обратно. И обрежем еще по `[0, 1]` (хотя в данных встречаются примеры, где все gt отрицательные, что с ними делать - не очень понятно):

In [18]:
results_dict = dict()

for idx in range(len(test_indices)):
    fi, si = test_indices[idx]
    di = data_packs[fi][si]
    im = di.image
    h, w = im.size    

    dx = max(0, (h - w) // 2)
    dy = max(0, (w - h) // 2)
    
    cur_pred = preds[idx, :].copy()

    cur_pred *= max(h, w) / 224.
    cur_pred[0] -= dy
    cur_pred[2] -= dy
    cur_pred[4] -= dy
    cur_pred[6] -= dy
    
    cur_pred[1] -= dx
    cur_pred[3] -= dx
    cur_pred[5] -= dx
    cur_pred[7] -= dx
    
    cur_pred[0] /= h
    cur_pred[2] /= h
    cur_pred[4] /= h
    cur_pred[6] /= h
    
    cur_pred[1] /= w
    cur_pred[3] /= w
    cur_pred[5] /= w
    cur_pred[7] /= w
    
    cur_pred = np.clip(cur_pred, 0, 1)
    
    results_dict[di.unique_key] = cur_pred.reshape(4, 2)

In [19]:
from course_intro_ocr_t1.metrics import dump_results_dict, measure_crop_accuracy

In [20]:
dump_results_dict(results_dict, Path() / 'pred.json')

In [21]:
acc = measure_crop_accuracy(
    Path() / 'pred.json',
    Path() / 'gt.json'
)

In [22]:
print(acc)

0.8063529411764706
